In [27]:
# http://domoticx.com/p1-poort-slimme-meter-telegram-uitlezen-met-python/
import re
import serial
from datetime import datetime
import pandas as pd

In [11]:

out = []
# Serial port config for DSMR 4.x
with serial.Serial(
    port="COM4",
    baudrate=115200,
    bytesize=8,
    parity="N",
    stopbits=1,
    timeout=5,
    xonxoff=False,
    rtscts=False,
) as ser:
    while True:
        telegram_line = ser.readline()
        out.append(telegram_line.decode('ascii').strip())

        if re.match(b'(?=!)', telegram_line):
            break

out

['/XMX5LGF0000453897791',
 '',
 '1-3:0.2.8(50)',
 '0-0:1.0.0(230423220911S)',
 '0-0:96.1.1(4530303531303035333839373739313139)',
 '1-0:1.8.1(004168.557*kWh)',
 '1-0:1.8.2(004391.605*kWh)',
 '1-0:2.8.1(000000.000*kWh)',
 '1-0:2.8.2(000000.000*kWh)',
 '0-0:96.14.0(0001)',
 '1-0:1.7.0(00.114*kW)',
 '1-0:2.7.0(00.000*kW)',
 '0-0:96.7.21(00013)',
 '0-0:96.7.9(00002)',
 '1-0:99.97.0(1)(0-0:96.7.19)(190708191813S)(0000000888*s)',
 '1-0:32.32.0(00008)',
 '1-0:32.36.0(00002)',
 '0-0:96.13.0()',
 '1-0:32.7.0(232.1*V)',
 '1-0:31.7.0(000*A)',
 '1-0:21.7.0(00.114*kW)',
 '1-0:22.7.0(00.000*kW)',
 '0-1:24.1.0(003)',
 '0-1:96.1.0(4730303339303031393433333238393139)',
 '0-1:24.2.1(230423220509S)(04824.432*m3)',
 '!5C66']

In [2]:
example_msg = [
    '/XMX5LGF0000453897791',
    '1-3:0.2.8(50)',
    '0-0:1.0.0(230423220831S)',
    '0-0:96.1.1(4530303531303035333839373739313139)',
    '1-0:1.8.1(004168.556*kWh)',
    '1-0:1.8.2(004391.605*kWh)',
    '1-0:2.8.1(000000.000*kWh)',
    '1-0:2.8.2(000000.000*kWh)',
    '0-0:96.14.0(0001)',
    '1-0:1.7.0(00.118*kW)',
    '1-0:2.7.0(00.000*kW)',
    '0-0:96.7.21(00013)',
    '0-0:96.7.9(00002)',
    '1-0:99.97.0(1)(0-0:96.7.19)(190708191813S)(0000000888*s)',
    '1-0:32.32.0(00008)',
    '1-0:32.36.0(00002)',
    '0-0:96.13.0()',
    '1-0:32.7.0(231.9*V)',
    '1-0:31.7.0(000*A)',
    '1-0:21.7.0(00.118*kW)',
    '1-0:22.7.0(00.000*kW)',
    '0-1:24.1.0(003)',
    '0-1:96.1.0(4730303339303031393433333238393139)',
    '0-1:24.2.1(230423220509S)(04824.432*m3)',
    '!65BD'
]

In [5]:
mapper = pd.read_excel('Definitions.xlsx', 'Sheet1', index_col='OBIS reference')
mapper.index = mapper.index.map(lambda x: x.replace('.255', ''))

In [28]:
def transform(tel):
    """Converts tel to a list of key-value pairs"""
    try:
        trans = [x for x in mapper.index if x in tel][0]
        name = mapper.loc[[trans], 'Short name'].values
    except (ValueError, IndexError):
        return list()

    tel = re.findall(r'\((.*?)\)', tel)
    ntel = []
    for t in tel:
        t = t.split('*')
        if len(t) == 1:
            t += ['-']
        ntel.append(t)
    
    return list(zip(name, *zip(*ntel)))

df = pd.DataFrame([x for item in example_msg for x in transform(item)])\
    .rename({0: 'Name', 1: 'Value', 2: 'Unit'}, axis=1)\
    .set_index('Name')

dts = [x for x in df.index if 'DateTime' in x]
dtv = [datetime.strptime(x.replace('S', ''), r'%Y%m%d%H%M%S') for x in df.loc[dts, 'Value'].values]
for s, v in zip(dts, dtv):
    df.loc[s, 'Value'] = v

df

,Value,Unit
Name,,
P1Version,50,-
DateTimeElectric,2304-02-03 22:08:31,-
EquipmentIDElectric,4530303531303035333839373739313139,-
ElectricityToClient(1),004168.556,kWh
ElectricityToClient(2),004391.605,kWh
ElectricityByClient(1),000000.000,kWh
ElectricityByClient(2),000000.000,kWh
TariffIndicator,0001,-
ActualElectricityToClient,00.118,kW
